In [1]:
#import modules
import pandas as pd
import numpy as np 
import seaborn as sb
import pandas_profiling
import matplotlib.pyplot as plt
import sqlalchemy as db
from sqlalchemy import MetaData, Table
from pandas_profiling import ProfileReport

In [2]:
#connecting to database
engine = db.create_engine("sqlite:///im.db")

In [3]:
#getting all table names
inspector = db.inspect(engine)
table_names = inspector.get_table_names()

In [4]:
metadata = MetaData()
connection = engine.connect()
all_data = []

#create for loop to get lists from tables
for i in table_names:
    t = Table(i, metadata, autoload = True, autoload_with=engine )
    query = db.select([t])
    ResultProxy = connection.execute(query)
    ResultSet = ResultProxy.fetchall()
    all_data.append(ResultSet)

In [5]:
# creating dataframes
directors = pd.DataFrame(all_data[0])
known_for = pd.DataFrame(all_data[1])
movie_akas = pd.DataFrame(all_data[2])
movie_basics = pd.DataFrame(all_data[3])
movie_ratings = pd.DataFrame(all_data[4])
persons = pd.DataFrame(all_data[5])
principals = pd.DataFrame(all_data[6])
writers = pd.DataFrame(all_data[7])


In [ ]:
#change person_id to diector_id
directors.columns = ["movie_id", "director_id"]
#change person_id to writer_id
writers.columns = ["movie_id", "writer_id"]

movie_directors_writers = directors.merge(writers).drop_duplicates(subset="movie_id")

In [98]:
movie_directors_writers.head()

,movie_id,director_id,writer_id
0,tt0285252,nm0899854,nm0899854
1,tt0462036,nm1940585,nm1940585
2,tt0835418,nm0151540,nm0310087
3,tt0835418,nm0151540,nm0841532
4,tt0835418,nm0151540,nm0310087


In [19]:
# directors_writers.duplicates

In [73]:
#filter out repeating columns
akas = movie_akas[['movie_id', 'ordering', 'region', 'language', 'types','attributes']]

In [12]:
#joining movie_ratings and movie_basics columns
basics_ratings = pd.merge(left=movie_basics, right=movie_ratings, left_on='movie_id', right_on='movie_id')

In [72]:
#joining basics_ratings and movie akas
br_akas = (basics_ratings.merge(akas, on="movie_id")).drop_duplicates(subset='movie_id', keep="first")
br_akas

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,ordering,region,language,types,attributes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,1,IN,hi,None,alternative transliteration
5,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,1,XWW,en,None,None
9,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,10,VE,None,None,None
22,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13,1,IN,bn,None,None
25,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119,1,None,None,original,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
261792,tt9899860,Watching This Movie Is a Crime,Didan in film jorm ast,2019,100.0,"Drama,Thriller",8.1,7,1,XWW,en,imdbDisplay,None
261795,tt9899880,Columbus,Columbus,2018,85.0,Comedy,5.8,5,1,None,None,original,None
261797,tt9903952,BADMEN with a good behavior,BADMEN with a good behavior,2018,87.0,"Comedy,Horror",9.2,5,1,DE,None,None,None
261800,tt9905462,Pengalila,Pengalila,2019,111.0,Drama,8.4,600,1,None,None,original,None


In [27]:
#change column names of person
persons.columns = ['person_id', 'person_name', 'person_birth_year', 'person_death_year', 'person_primary_profession']

#change column names of principals
principals.columns = ['movie_id', 'ordering', 'person_id', 'person_category', 'person_job', 'person_characters']

#merging the two dataframes
principal_person = principals.merge(persons)

Index(['person_id', 'person_name', 'person_birth_year', 'person_death_year',
       'person_primary_profession'],
      dtype='object')

In [88]:
#merge known for movie
known_for.columns = ['person_id', 'known_movie_id']

known_for.shape
famous = known_for.merge(principal_person, on="person_id").drop_duplicates(subset="person_id", keep="first")

In [91]:
famous.isnull().sum()

person_id                         0
known_movie_id                    0
movie_id                          0
ordering                          0
person_category                   0
person_job                   478503
person_characters            342963
person_name                       0
person_birth_year            491856
person_death_year            567567
person_primary_profession     41298
dtype: int64

In [1]:
## Importing data
movie_gross = pd.read_csv("./movies/bom.movie_gross.csv")

NameError: name 'pd' is not defined